In [2]:
from psaw import PushshiftAPI                               #Importing wrapper library for reddit(Pushshift)
import datetime as dt                                       #Importing library for date management
import pandas as pd                                         #Importing library for data manipulation in python
import matplotlib.pyplot as plt      
import yfinance as yf                    #Importing library for creating interactive visualizations in Python

api = PushshiftAPI()  #establishing the API and parameters

What I have so far:
- data set within a specific time frame that pulls in all comments that reference a key word
- decided what metric to use to track 'frequency' --> total number of upvotes on all posts/ comments?
- created a data frame by collapsing the reddit data frame onto a daily basis
- created a data frame that tracks stock prices over time using y finance
- merged the two data sets
- create similar data frame for posts

Current Challenges:
- 5000 comments only returns 15 entries (days) which is only 15 data points for the regression. What should I do?
- How do I turn the data set merging into a function? I'm getting errors

In [3]:
      


#epoch time format is the number of seconds since 00:00:00 UTC on Jan 1, 1970.
#The datetime method accepts a date (year, month, day, hour, minute, second, microsecond)

before = int(dt.datetime(2021,2,26,0,0).timestamp()) #.timestamp() converts the date to epoch time 
after = int(dt.datetime(2021,2,3,0,0).timestamp())

subreddit="wallstreetbets"
limit=5000
word_to_check = 'gme to the moon'



In [18]:
#Creating a data frame with comment data



comments = api.search_comments(q= word_to_check, subreddit=subreddit, limit=limit, before=before, after=after)

comments_list=[]


for comment in comments:
    #if word_to_check in comment.body.lower():#case insensitive check
        comments_list.append(
            {"comment_id": comment.id, "comment_text": comment.body,"score": comment.score, "date":comment.created}
        )

comments_df=pd.DataFrame(comments_list)


comments_df["date"] = pd.to_datetime(comments_df["date"], unit="s").dt.date

#pd.set_option("display.max_colwidth", -1) #this is how to view full comments text

display(comments_df)

/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


,comment_id,comment_text,score,date
0,got3lwc,"Germany gme up 20% this morning, get ready to ...",2,2021-02-26
1,got3dbl,Nooooooo. Look the echo chamber here might te...,3,2021-02-26
2,got3biy,ELON tweeted last night AGAIN. It was an SN ro...,1,2021-02-26
3,got27je,$AMC &amp; $GME to the moon💎💎🦍🦍,12,2021-02-26
4,got22oq,I had lost all emotion on my -70% decline of m...,91,2021-02-26
...,...,...,...,...
4354,glu9886,Not when you're buying GME. Gamestonks to the ...,2,2021-02-03
4355,glu8ze0,Enough. It’s over. Literally money to be made ...,1,2021-02-03
4356,glu8wsu,"No point in selling GME at such heavy loss, ma...",9,2021-02-03
4357,glu8urw,Moves? I will hold my only GME until end or th...,14,2021-02-03


In [4]:
#Collapsing to net score per day for given phrase

def Reddit_Comment_Data (q, subbreddit, limit, before, after):

    comments = api.search_comments(q= word_to_check, subreddit= subreddit, limit= limit, before=before, after=after)
    comments_list=[]

    for comment in comments:
        comments_list.append(
            {"score": comment.score, "Date":comment.created}
        )

    comments_df=pd.DataFrame(comments_list)

    comments_df["Date"] = pd.to_datetime(comments_df["Date"], unit="s").dt.date

    grouped_date_df = comments_df.groupby(by="Date")["score"].sum()

    pd.DataFrame(grouped_date_df)

    grouped_date_df = grouped_date_df.reset_index()

    grouped_date_df['Date'] = grouped_date_df['Date'].astype(str)

    return grouped_date_df


In [9]:
Reddit_Comment_Data('amc', subreddit, limit, before, after)

/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


,Date,score
0,2021-02-03,5745
1,2021-02-04,2568
2,2021-02-08,156
3,2021-02-09,851
4,2021-02-10,2969
5,2021-02-11,1444
6,2021-02-12,1386
7,2021-02-13,179
8,2021-02-14,69
9,2021-02-15,50


In [16]:
#Creating a stock price data frame

def Stock_Price_Data(ticker, after, before):
    data = yf.download(ticker,after,before)

    stock_price_df = pd.DataFrame(data)
    stock_price_df = stock_price_df.reset_index() #making Date a column rather than an index
    stock_price_df['Date'] = stock_price_df['Date'].astype(str) #convert Date to string format to prepare for merge
    
    return stock_price_df


In [17]:
Stock_Price_Data("GME", after, before)

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-02-03,28.002501,28.350000,21.3125,23.102501,23.102501,170794000
1,2021-02-04,22.797501,22.875000,13.3325,13.375000,13.375000,249709200
2,2021-02-05,13.510000,23.750000,12.7725,15.942500,15.942500,325380000
3,2021-02-08,18.102501,18.165001,14.5050,15.000000,15.000000,102749200
4,2021-02-09,14.152500,14.250000,11.6300,12.577500,12.577500,107372400
5,2021-02-10,12.692500,15.707500,11.6375,12.800000,12.800000,145820000
6,2021-02-11,12.502500,13.830000,12.0550,12.775000,12.775000,52226800
7,2021-02-12,12.687500,13.810000,12.0125,13.100000,13.100000,58293200
8,2021-02-16,13.165000,13.375000,12.2600,12.377500,12.377500,32700000
9,2021-02-17,12.442500,12.797500,11.1400,11.485000,11.485000,36747200


In [18]:
#Merging Reddit Data with Stock Price Data

#mergedData = pd.merge(stock_price_df, grouped_date_df, on='Date') #this worked before I created functions

mergedData = pd.merge(Reddit_Comment_Data, Stock_Price_Data, on='Date')


del mergedData["Adj Close"]
del mergedData["Open"]
del mergedData["High"]
del mergedData["Low"]

mergedData

TypeError: Can only merge Series or DataFrame objects, a <class 'function'> was passed

In [96]:
#regression analysis

correlation = mergedData.corr()

correlation.loc['Close', 'score']

0.8030475878600717

In [10]:
#Creating a data frame with post data

posts = api.search_submissions(q= word_to_check, subreddit = subreddit, limit = 20, start_time = after, end_time = before)
#posts = api.search_submissions(limit=10)

posts_list=[]

for post in posts:
    posts_list.append(
        {"post_id": post.id, "post_title": post.title, "post_text": post.selftext,"score": post.score} 
    )


posts_df=pd.DataFrame(posts_list)


#posts_df["date"] = pd.to_datetime(posts_df["date"], unit="s").dt.date


display(posts_df)

/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


,post_id,post_title,post_text,score
0,ywt7cx,"Apes, it’s time! AMC, APE and GME to the moon!...",,1
1,yrnrdt,Time for a second movement?,Obviously the first round went well and we sho...,1
2,yqrdtm,"After holding GME since Feb 21, I finally thro...",,1
3,ypu9oj,Setting Wallstreet on fire with $UPST?,I found this very interesting stock $UPST (Ups...,1
4,yotviv,US Natural Gas to the moon!,this post is probably going to be downvoted ev...,1
5,ymtqhk,GME TO THE MOON I AM A TIME TRAVELER HEED MY W...,,1
6,yjpeu7,What are your predictions on GME. Is it going ...,,1
7,yiqds7,1.. 2.. 3.. GME TO THE MOON 🚀,,1
8,yict4a,Let’s RUN GME to the moon,,1
9,yics1n,They don’t want us to go to the MOON. #gme,,1
